# XGBoost

## References

* [Chen et al. 2016, XGBoost: A Scalable Tree Boosting System](https://dl.acm.org/doi/10.1145/2939672.2939785)

## The core algorithm

### Training

1. make a simple baseline estimate, e.g. average of `y`
2. compute the differences of `y` and the baseline -> `dy`
3. while `dy` values are too large:
    * train new model predicting `dy`
    * update remaining `dy`

So at the end you have a baseline estimate and a bunch of models correcting the prediction for each observation. 

For the actual algorithms one needs to define a loss to be minimized to know what `dy` exactly, see the _Algorithm 1 Gradient_Boost_ in Friedman et al. 2001.

### Inference

1. for each model compute its prediction
2. sum baseline estimate and model predictions

## XGBoost loss

$$ \text{loss}^{(t)} = \sum_{i=1}^n l(\text{target}_i, \text{cumulative estimate}^{(t-1)}_i + \text{change in estimate}^{(t)}_i ) + \Omega\left(\text{new estimation function}^{(t)}\right)$$

$t$ is the boosting iteration / tree number and $i$ is a single observation.

Let's use
* $ \text{new estimation function}^{(t)} = f_t $
* $ \text{change in estimate}^{(t)}_i = f_t(x_i)$
* $ \text{target}_i = y_i$
* $ \text{cumulative estimate}^{(t-1)}_i = \hat{y}^{(t-1)}_i$


Then we can shorten the $\text{loss} ^ {(t)}$ description to a more cryptic
$$ \text{loss}^{(t)} = \sum_{i=1}^n l(y_i, \hat{y}^{(t-1)}_i + f^{(t)}(x_i) ) + \Omega\left(f^{(t)}\right) $$ 

For the regularization the authors use
$$ \Omega (f_t) = \gamma N^{(t)}_\text{leafs} + \frac{1}{2} \lambda \sum^{N^{(t)}_\text{leafs}}_j w_j^2$$

where $\gamma$ is some constant and $w_j$ is a leaf weight (seems like the $\gamma_{jm}$ from Friedman et al. but isn't clarified)

Approximating how $l(y_i, \hat{y}^{(t-1)}_i + f^{(t)}(x_i) )$ changes for varying $f$, with a 2nd order Taylor polynomial, the authors write
$$ \text{loss}^{(t)} \approx \sum_{i=1}^n l(y_i, \hat{y}^{(t-1)}_i)  + g_i f^{(t)}(x_i) + \frac{1}{2} h_i \left(f^{(t)}(x_i)\right)^2   + \Omega\left(f^{(t)}\right) $$

where $g_i = \frac{\partial l(y_i, \hat{y}^{(t-1)}_i)}{\partial \hat{y}^{(t-1)}}$ and $h_i = \frac{\partial^2 l(y_i, \hat{y}^{(t-1)}_i)}{\partial \left(\hat{y}^{(t-1)}\right)^2}$ contain the 1st and 2nd partial derivatives of the loss function $l$. 

A key property of a tree is that every observation $x_i$ is part of exactly one leaf $I_j$, and hence exactly one predicted value / leaf weight $w_j$, or in cryptic
$$ f^{(t)}(x_i) = w_j \space \forall x_i \in I_j$$

Using this aspect and the loss above the authors derive the optimal leaf values $w_j^*$ as 

$$ w_j^* = - \frac{\sum_{i \in I_j} g_i}{\sum_{i \in I_j} h_i + \lambda} $$

where $I_j$ is the set of observations / **I**nstances in leaf $j$

And the loss for the new tree (ignoring the cumulative part $l(y_i, \hat{y}^{(t-1)}_i)$ which is not influenced by the optimization of $\text{loss}^{(t)}$) is dervied as

$$ \text{loss new tree}^{(t)} \approx - \frac{1}{2} \sum_{j=1}^{N_\text{leafs}} \frac{\left( \sum_{i \in I_j} g_i \right)^2}{\sum_{i \in I_j} h_i + \lambda} + \gamma N_\text{leafs} $$

So if we contemplate to introduce another split ($N_\text{leafs, new} = N_\text{leafs} + 1$) we have the difference of $\text{loss new tree with another split}^{(t)} - \text{loss new tree}^{(t)}$

$$  = \left(- \frac{1}{2} \sum_{j=1}^{N_\text{leafs}+1} \frac{\left( \sum_{i \in I_{\text{new},j}} g_i \right)^2}{\sum_{i \in I_{\text{new},j}} h_i + \lambda} + \gamma \left(N_\text{leafs} + 1\right) \right) - \left( - \frac{1}{2} \sum_{j=1}^{N_\text{leafs}} \frac{\left( \sum_{i \in I_j} g_i \right)^2}{\sum_{i \in I_j} h_i + \lambda} + \gamma N_\text{leafs} \right) $$

$$  = - \frac{1}{2} \left( \frac{\left( \sum_{i \in I_{\text{left}}} g_i \right)^2}{\sum_{i \in I_{\text{left}}} h_i + \lambda} + \frac{\left( \sum_{i \in I_{\text{right}}} g_i \right)^2}{\sum_{i \in I_{\text{right}}} h_i + \lambda} \right) + \gamma + \frac{1}{2} \frac{\left( \sum_{i \in I} g_i \right)^2}{\sum_{i \in I} h_i + \lambda} $$

$$  = - \frac{1}{2} \left( \frac{\left( \sum_{i \in I_{\text{left}}} g_i \right)^2}{\sum_{i \in I_{\text{left}}} h_i + \lambda} + \frac{\left( \sum_{i \in I_{\text{right}}} g_i \right)^2}{\sum_{i \in I_{\text{right}}} h_i + \lambda} - \frac{\left( \sum_{i \in I} g_i \right)^2}{\sum_{i \in I} h_i + \lambda} \right) + \gamma   $$

The three components in the round braces above is then what can be used to decide on whether to split the set of observations $I$ into $I_\text{left}$ and $I_\text{right}$ or not.

Let's look at example $g$ and $h$ values. 

Regression - Least squares $l(y_i, \hat{y}^{(t-1)}_i) = \frac{\left(y_i - \hat{y}^{(t-1)}_i\right)^2}{2}$:
* $g_i = \hat{y}^{(t-1)}_i - y_i$
* $h_i = 1$

Binary classification - negative binomial log-likelihood $l(y_i, \hat{y}^{(t-1)}_i) = \log\left(1 + \exp \left(-2 y_i \cdot \hat{y}^{(t-1)}_i\right)\right)$, where $y_i \in \{-1,1\}$ and $\hat{y}^{(t-1)}_i \in \mathbb{R}$:
* $g_i = - \frac{2 y_i}{(\exp(2 y_i \hat{y}^{(t-1)}_i) + 1)}$
* $h_i = \frac{4 y_i^2 \exp(2 y_i \hat{y}^{(t-1)}_i))}{(\exp(2 y_i \hat{y}^{(t-1)}_i) + 1)^2}$

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn.datasets as sk_datasets

import random_tree_models.decisiontree as dtree
import random_tree_models.gradientboostedtrees as gbtree
import random_tree_models.xgboost as xgboost

In [ ]:
rng = np.random.RandomState(42)

## Classification

In [ ]:
X, y = sk_datasets.make_classification(
    n_samples=1_000,
    n_features=2,
    n_classes=2,
    n_redundant=0,
    class_sep=2,
    random_state=rng,
)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, alpha=0.3);

In [ ]:
model = xgboost.XGBoostClassifier(
    measure_name="xgboost", max_depth=2, n_trees=3, lam=0.0
)

In [ ]:
model.fit(X, y)

In [ ]:
dtree.show_tree(model.trees_[0])

In [ ]:
y_prob = model.predict_proba(X)
y_prob[:5]

In [ ]:
x0 = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
x1 = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
X0, X1 = np.meshgrid(x0, x1)
X_plot = np.array([X0.ravel(), X1.ravel()]).T

In [ ]:
y_prob = model.predict_proba(X_plot)[:, 1]
y_prob[:5]

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolormesh(X0, X1, y_prob.reshape(X0.shape), alpha=0.2)
fig.colorbar(im, ax=ax)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, ax=ax, alpha=0.3)
plt.show()

## Regression

In [ ]:
X, y, coefs = sk_datasets.make_regression(
    n_samples=1_000, n_features=2, n_targets=1, coef=True, random_state=rng
)
sns.scatterplot(x=X[:, 0], y=y, alpha=0.3)

In [ ]:
model = xgboost.XGBoostRegressor(
    measure_name="xgboost", max_depth=2, n_trees=3, lam=0.0
)

In [ ]:
model.fit(X, y)

In [ ]:
dtree.show_tree(model.trees_[0])

In [ ]:
x0 = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
x1 = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
X0, X1 = np.meshgrid(x0, x1)
X_plot = np.array([X0.ravel(), X1.ravel()]).T

In [ ]:
y_pred = model.predict(X_plot)
y_pred[:5]

In [ ]:
fig, axs = plt.subplots(nrows=2, figsize=(12, 6))

ax = axs[0]
sns.scatterplot(x=X_plot[:, 0], y=y_pred, ax=ax, alpha=0.1, label="prediction")

ax = axs[1]
sns.scatterplot(x=X_plot[:, 1], y=y_pred, ax=ax, alpha=0.1, label="prediction")

plt.tight_layout()

In [ ]:
fig, ax = plt.subplots()
im = ax.pcolormesh(X0, X1, y_pred.reshape(X0.shape), alpha=0.2)
fig.colorbar(im, ax=ax)
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=y, ax=ax, alpha=0.3)
plt.show()

In [ ]:
y_pred = model.predict(X)

fig, axs = plt.subplots(nrows=2, figsize=(12, 6))

ax = axs[0]
sns.scatterplot(x=X[:, 0], y=y_pred, ax=ax, alpha=0.1, label="prediction")
sns.scatterplot(x=X[:, 0], y=y, ax=ax, alpha=0.1, label="actual")

ax = axs[1]
sns.scatterplot(x=X[:, 1], y=y_pred, ax=ax, alpha=0.1, label="prediction")
sns.scatterplot(x=X[:, 1], y=y, ax=ax, alpha=0.1, label="actual")

plt.tight_layout()